## Import libraries

In [1]:
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, Adadelta
from keras.models import model_from_json
from keras import backend as K
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

from sklearn.metrics import classification_report, confusion_matrix
from textblob import TextBlob

import numpy as np, string, pickle, warnings, random
import matplotlib.pyplot as plt

from IPython.display import display, HTML

from deepexplain.tensorflow import DeepExplain
import tensorflow as tf
warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\t

## Download data

In [8]:
topWords = 50000
maxWords = 200
downloadFlag = 0

df = pd.read_csv('./data/multidogoSentLevel/finance/train.tsv', sep='\t')
x_train = df['utterance']
y_train = df['intent']

df = pd.read_csv('./data/multidogoSentLevel/finance/dev.tsv', sep='\t')
x_val = df['utterance']
y_val = df['intent']

df = pd.read_csv('./data/multidogoSentLevel/finance/test.tsv', sep='\t')
x_test = df['utterance']
y_test = df['intent']

classes_train = set(y_train)
classes_val = set(y_val)
classes_test = set(y_test)
classes = classes_train.union(classes_val, classes_test)
nb_classes = len(classes)
sentimentDict = {}
for i, cl in enumerate(classes):
    sentimentDict[i] = cl
sentimentDict2 = {v: k for k, v in sentimentDict.items()}
print(sentimentDict)
y_train = [sentimentDict2[cl] for cl in list(y_train)]
y_val = [sentimentDict2[cl] for cl in list(y_val)]
y_test = [sentimentDict2[cl] for cl in list(y_test)]

data = x_train.append(x_val)
data = data.append(x_test)
vocab = []
for utter in data:
    for word in str(utter).split(' '):
        vocab.append(word)
vocab = set(vocab)
word2Index = {}
for i, word in enumerate(vocab):
    word2Index[word] = i
index2Word = {v: k for k, v in word2Index.items()}

train = []
for utter in x_train:
    utt = [word2Index[word] for word in str(utter).split(' ')]
    train.append(utt)
x_train = train
val = []
for utter in x_val:
    utt = [word2Index[word] for word in str(utter).split(' ')]
    val.append(utt)
x_val = val
test = []
for utter in x_test:
    utt = [word2Index[word] for word in str(utter).split(' ')]
    test.append(utt)
x_test = test

{0: 'closeaccount', 1: 'checkbalance<div>orderchecks', 2: 'reportlostcard', 3: 'confirmation', 4: 'openinggreeting', 5: 'checkoffereligibility', 6: 'disputecharge<div>reportlostcard', 7: 'contentonly', 8: 'disputecharge', 9: 'closinggreeting', 10: 'updateaddress', 11: 'thankyou', 12: 'openaccount', 13: 'getroutingnumber', 14: 'checkbalance<div>transfermoney', 15: 'transfermoney', 16: 'closeaccount<div>reportlostcard', 17: 'replacecard', 18: 'replacecard<div>reportlostcard', 19: 'requestcreditlimitincrease', 20: 'outofdomain', 21: 'closeaccount<div>transfermoney', 22: 'checkbalance', 23: 'rejection', 24: 'orderchecks'}


In [9]:
print(data)

0                                                hi
1                     report an my credit card lost
2                                            stphon
3                                             edson
4                                              4577
                           ...                     
4416    i need transfer money my ac to my friend ac
4417                                  arun ssn 6058
4418                                           5987
4419                           ok thank you so much
4420                                  no thank byee
Name: utterance, Length: 22018, dtype: object


In [10]:
stopWords = []

def getWordsFromIndexList(indexList):
    wordList = []
    for index in indexList:
        wordList.append(index2Word[index])

    return " ".join(wordList)

def getSentiment(predictArray):
    pred = int(predictArray[0])
    return sentimentDict[pred]

def getIndexFromWordList(wordList):
    indexList = []
    for word in wordList:
        print(word)
        indexList.append(str(word2Index[word]))
        
    return indexList

In [11]:
print (len(word2Index))

8258


## Preprocess data

In [12]:
stopIndexList = []

# for stopWord in stopWords:
#     stopIndexList.append(word2Index[stopWord])

trainData = []

for indexList in x_train:
    processedList = [index for index in indexList]# if index not in stopIndexList]
    trainData.append(processedList)
    
x_train = trainData

## Data Pre-processing

In [13]:
'''
Padding data to keep vectors of same size
If size < 100 then it will be padded, else it will be cropped
'''
trainX = pad_sequences(x_train, maxlen = maxWords, value = 0.)
valX = pad_sequences(x_val, maxlen = maxWords, value = 0.)
testX = pad_sequences(x_test, maxlen = maxWords, value = 0.)

'''
One-hot encoding for the classes
'''
trainY = np_utils.to_categorical(y_train, num_classes = nb_classes)
valY = np_utils.to_categorical(y_val, num_classes = nb_classes)
testY = np_utils.to_categorical(y_test, num_classes = nb_classes)

## Network Parameters

In [14]:
p_W, p_U, weight_decay = 0, 0, 0
regularizer = l2(weight_decay) if weight_decay else None
sgdOptimizer = 'adam'
lossFun='categorical_crossentropy'
batchSize=25
numEpochs = 5
numHiddenNodes = 256

## Network Architecture

In [15]:
model = Sequential()
model.add(Embedding(topWords, numHiddenNodes, name='embedding_layer'))

model.add(Bidirectional(LSTM(numHiddenNodes, return_sequences=True,
                            recurrent_regularizer=regularizer, kernel_regularizer=regularizer,
                            bias_regularizer=regularizer, recurrent_dropout=p_W, dropout=p_U),
                        merge_mode='concat', name='bidi_lstm_layer'))
model.add(LSTM(numHiddenNodes, name = 'lstm_layer'))
model.add(Dropout(0.5, name = 'dropout'))

model.add(Dense(numHiddenNodes, name='dense_1'))
model.add(Dense(nb_classes, name='dense_2'))
model.add(Activation("softmax"))
# adam = Adam(lr=0.0001)
#adadelta = Adadelta(lr=0.01, rho=0.95, epsilon=1e-08)
model.compile(loss=lossFun, optimizer=sgdOptimizer, metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 256)         12800000  
_________________________________________________________________
bidi_lstm_layer (Bidirection (None, None, 512)         1050624   
_________________________________________________________________
lstm_layer (LSTM)            (None, 256)               787456    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 25)                6425      
_________________________________________________________________
activation_1 (Activation)    (None, 25)                0         
Total para

## Training the model

In [16]:
model.fit(trainX, trainY, batch_size=batchSize, epochs=numEpochs, verbose=1, validation_data=(valX, valY))

Train on 15388 samples, validate on 2209 samples
Epoch 1/5
15388/15388 [==============================] - 1116s - loss: 0.8243 - acc: 0.7716 - val_loss: 0.4104 - val_acc: 0.8972
Epoch 2/5
15388/15388 [==============================] - 1246s - loss: 0.3989 - acc: 0.8957 - val_loss: 0.3919 - val_acc: 0.9013
Epoch 3/5
15388/15388 [==============================] - 1583s - loss: 0.3149 - acc: 0.9175 - val_loss: 0.3948 - val_acc: 0.9022
Epoch 4/5
15388/15388 [==============================] - 2061s - loss: 0.2711 - acc: 0.9285 - val_loss: 0.3714 - val_acc: 0.9077
Epoch 5/5
15388/15388 [==============================] - 1544s - loss: 0.2527 - acc: 0.9334 - val_loss: 0.3925 - val_acc: 0.9095


# Model accuracy

In [17]:
score = model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

acc: 89.39%


In [18]:
predY = model.predict_classes(testX)
yPred = np_utils.to_categorical(predY, num_classes = nb_classes)
print("Classification Report:\n")
print(classification_report(testY, yPred))

4421/4421 [==============================] - 72s    
Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.79      0.84        24
           1       0.00      0.00      0.00         0
           2       0.88      0.92      0.90       117
           3       0.89      0.93      0.91       535
           4       0.99      0.96      0.97       383
           5       0.00      0.00      0.00        16
           6       0.00      0.00      0.00         0
           7       0.97      0.93      0.95      1652
           8       0.74      0.82      0.77        65
           9       0.87      0.80      0.83       133
          10       0.86      0.89      0.88       104
          11       0.96      0.90      0.93       574
          12       0.00      0.00      0.00         0
          13       0.80      0.24      0.36        17
          14       0.00      0.00      0.00         2
          15       0.82      0.86      0.84        72
    

## Save the Tensorflow Model

In [16]:
'''
Serialize model to JSON
'''
model_json = model.to_json()
with open("models/imdb_bi_lstm_big_tensorflow_model_ALL.json", "w") as json_file:
    json_file.write(model_json)

'''
Serialize weights to HDF5
'''
model.save_weights("models/imdb_bi_lstm_big_tensorflow_model_ALL.h5", overwrite=True)
print("Saved model to disk...")

FileNotFoundError: [Errno 2] No such file or directory: 'models/imdb_bi_lstm_big_tensorflow_model_ALL.json'

## Load the Tensorflow Model

In [ ]:
'''
Load json and create model
'''
modelNum=1
if modelNum is 1:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model.json', 'r')
elif modelNum is 2:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model_NOT.json', 'r')
elif modelNum is 3:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model_ALL.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

'''
Load weights into new model
'''
if modelNum is 1:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model.h5")
elif modelNum is 2:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model_NOT.h5")
elif modelNum is 3:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model_ALL.h5")

print ("Loading model from disk...")
model = loaded_model
print(model.summary())

## Testing the model

In [19]:
num = 120
num_next = num + 1
print("Testing for test case..." + str(num))
groundTruth = testY[num]
print(groundTruth)
print(len(testY))
print

sampleX = testX[num:num_next]
predictionClass = model.predict_classes(sampleX, verbose=0)
prediction = np_utils.to_categorical(predictionClass, num_classes = nb_classes)[0]

print("Text: " + str(getWordsFromIndexList(x_test[num-1])))
print("\nPrediction: " + str(getSentiment(predictionClass)))
if np.array_equal(groundTruth,prediction):
    print("\nPrediction is Correct")
else:
    print("\nPrediction is Incorrect")

Testing for test case...120
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
4421
Text: ok thank you so much

Prediction: rejection

Prediction is Incorrect


# Explainability

## Get Relevance for each word

In [20]:
def getWordsAndRelevances(sampleX, relevances, prediction):

    unknownIndex = 0 # Index of padding
    indexList = np.ndarray.tolist(sampleX)[0]
    wordList = []
    wordRelevanceList = []
    polarityHighThreshold = 0.4
    polarityLowThreshold = 0.0
    
    # Find word-wize relevance and normalize
    wordRelevances = np.sum(relevances, -1)
    wordRelList = np.ndarray.tolist(wordRelevances)[0]
    
    for i in range(maxWords):
        index = indexList[i]
        relevance = wordRelList[i]

        if index is not unknownIndex:
            word = index2Word[index]
            blobText = TextBlob(word)
            polarity = blobText.sentiment.polarity
            prediction = int(prediction)
#             if prediction is 0 and polarity < -0.4 or prediction is 1 and polarity > 0.4:
            if abs(polarity) > polarityHighThreshold:
                if prediction is 0 and polarity < 0 or prediction is 1 and polarity > 0:
                    relevance = abs(polarity)
                elif prediction is 0 and polarity > 0 or prediction is 1 and polarity < 0:
                    relevance = -1 * abs(polarity)
                    
                print(word, polarity)
                print(word, relevance)
            elif abs(polarity) > polarityLowThreshold:
                if relevance < 0 and polarity > 0 or relevance > 0 and polarity < 0:
                    relevance = polarity
#                     print("Here...")
#                     print(word, polarity)
#                     print(word, relevance)

            wordList.append(index2Word[index])
            wordRelevanceList.append(relevance)

    
    normalizedRelevanceList = [float(rel)/max(map(lambda x: abs(x), wordRelevanceList)) for rel in wordRelevanceList]
    
    return wordList, wordRelevanceList, normalizedRelevanceList

def showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList):

    print("\nWord Relevances:\n")
    print("\nOriginal Relevance:\n")
    for i in range(len(wordList)):
        word = str(wordList[i])
        # originalRelevance = "{:8.2f}".format(wordRelevanceList[i])
        originalRelevance = wordRelevanceList[i]
        print ("\t\t\t" + str(originalRelevance) + "\t" + word)

    print("\nNormalized Relevance:\n")
    for i in range(len(wordList)):
        word = str(wordList[i])
        normalizedRelevance = "{:8.2f}".format(normalizedRelevanceList[i])
        print ("\t\t\t" + str(normalizedRelevance) + "\t" + word)
        

## Pre-process and get prediction

In [21]:
def removePunct(inputStr):
    
    # Remove punctuations.
    # punctuations = string.punctuation
    # inputStr = ''.join(ch for ch in inputStr if ch not in punctuations)
    inputStr = inputStr.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    inputList = inputStr.lower().split()
    return inputList

def removeStopWords(inputList):
    processedList = [word for word in inputList if word not in stopWords]
    return processedList

def preProcessQuery(inputStr):
    return removeStopWords(removePunct(inputStr))
#     return removePunct(inputStr)

def getIndexArray(inputStr):
    
    words = preProcessQuery(inputStr)
    # print(words)
    wordIndexList = np.array([word2Index[word] if word in word2Index else 0 for word in words])
    wordIndexArray = np.array([wordIndexList], np.int32)
    wordIndexArray = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)
    return wordIndexArray

def getPrediction(inputStr):
    
    wordIndexArray = getIndexArray(inputStr)
    predictionScore = model.predict(wordIndexArray[0:1], verbose=0)
    # print (predictionScore)
    prediction = model.predict_classes(wordIndexArray[0:1], verbose=0)
    return prediction, predictionScore

## Get Polarity from input text

In [22]:
inputStr = input("Enter a sentence to test polarity: ")
prediction, predictionScore = getPrediction(inputStr)
print("\nPrediction: " + str(getSentiment(prediction)))
print(predictionScore)

Enter a sentence to test polarity: ok thank you so much

Prediction: thankyou
[[4.7433574e-07 1.6690846e-07 8.8017096e-06 2.2677958e-03 3.4488278e-06
  1.0533725e-05 3.9662481e-07 2.1514213e-06 5.7859449e-07 7.8452147e-05
  7.4866880e-06 9.9732506e-01 1.2013827e-07 8.7638255e-06 6.2473106e-07
  2.5229156e-06 1.6369023e-06 3.9780636e-07 7.3433812e-06 6.1432098e-07
  3.4002242e-05 1.0027171e-06 9.2897625e-07 2.3648514e-04 2.1977860e-07]]


# Relevance and HeatMap

## Create DeepExplain Session to get word relevances

In [23]:
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    
    '''
    Need to reconstruct the graph in DeepExplain context, using the same weights.
    1. Get the input tensor
    2. Get embedding tensor
    3. Target the output of the last dense layer (pre-softmax)
    '''
    
    inputTensor = model.layers[0].input
    embeddingTensor = model.layers[0].output
    preSoftmax = model.layers[-2].output
    
    # Sample Data for attribution
    wordIndexArray = getIndexArray(inputStr)
    sampleX = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)
    
    # Perform Embedding Lookup
    getEmbeddingOutput = K.function([inputTensor],[embeddingTensor])
    embeddingOutput = getEmbeddingOutput([sampleX])[0]
    
    # Get Prediction for attribution
    prediction, predictionScore = getPrediction(inputStr)
    ys = np_utils.to_categorical(prediction, num_classes = nb_classes)
    
    #relevances = de.explain('grad*input', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('saliency', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('intgrad', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('deeplift', targetTensor * ys, inputTensor, sampleX)
    relevances = de.explain('elrp', preSoftmax * ys, embeddingTensor, embeddingOutput)
    #relevances = de.explain('occlusion', targetTensor * ys, inputTensor, sampleX)
    print ("Feature Relevances Generated...")

Feature Relevances Generated...


## Show Relevance

In [24]:
wordList, wordRelevanceList, normalizedRelevanceList = getWordsAndRelevances(sampleX, relevances, prediction[0])
showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList)

ok 0.5
ok -0.9503447413444519

Word Relevances:


Original Relevance:

			-0.9503447413444519	ok
			1.477203369140625	thank
			0.377288281917572	you
			0.19523566961288452	so
			0.2	much

Normalized Relevance:

			   -0.64	ok
			    1.00	thank
			    0.26	you
			    0.13	so
			    0.14	much


## Generate Heatmap

In [25]:
def rescale_score_by_abs (score, max_score, min_score):
    """
    rescale positive score to the range [0.5, 1.0], negative score to the range [0.0, 0.5],
    using the extremal scores max_score and min_score for normalization
    """
    
    # CASE 1: positive AND negative scores occur --------------------
    if max_score>0 and min_score<0:
    
        if max_score >= abs(min_score):   # deepest color is positive
            if score>=0:
                return 0.5 + 0.5*(score/max_score)
            else:
                return 0.5 - 0.5*(abs(score)/max_score)

        else:                             # deepest color is negative
            if score>=0:
                return 0.5 + 0.5*(score/abs(min_score))
            else:
                return 0.5 - 0.5*(score/min_score)   
    
    # CASE 2: ONLY positive scores occur -----------------------------       
    elif max_score>0 and min_score>=0: 
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5*(score/max_score)
    
    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score<=0 and min_score<0: 
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5*(score/min_score)    
  
      
def getRGB (c_tuple):
    return "#%02x%02x%02x"%(int(c_tuple[0]*255), int(c_tuple[1]*255), int(c_tuple[2]*255))

     
def span_word (word, score, colormap):
    return "<span style=\"background-color:"+getRGB(colormap(score))+"\">"+word+"</span>"


def html_heatmap (words, scores, cmap_name="bwr"):
    
    colormap  = plt.get_cmap(cmap_name)
     
    assert len(words)==len(scores)
    max_s     = max(scores)
    min_s     = min(scores)
    
    output_text = ""
    
    for idx, w in enumerate(words):
        score       = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, colormap) + " "

    return output_text + "\n"
            
relevanceScores = [-1.00 * scr for scr in wordRelevanceList]
htmlHeatMap = html_heatmap(wordList, relevanceScores)
print ("\nLRP heatmap:")    
display(HTML(htmlHeatMap))


LRP heatmap:


In [26]:
for i, inputStr in enumerate(x_test):
    inputStr = str(getWordsFromIndexList(inputStr))
    prediction, predictionScore = getPrediction(inputStr)
    if str(getSentiment(prediction)) == sentimentDict[y_test[i]]:
        pass
    else :
        print('Prediction = ', str(getSentiment(prediction)))
        print('Actual = ',sentimentDict[y_test[i]])
        with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context

            '''
            Need to reconstruct the graph in DeepExplain context, using the same weights.
            1. Get the input tensor
            2. Get embedding tensor
            3. Target the output of the last dense layer (pre-softmax)
            '''

            inputTensor = model.layers[0].input
            embeddingTensor = model.layers[0].output
            preSoftmax = model.layers[-2].output

            # Sample Data for attribution
            wordIndexArray = getIndexArray(inputStr)
            sampleX = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)

            # Perform Embedding Lookup
            getEmbeddingOutput = K.function([inputTensor],[embeddingTensor])
            embeddingOutput = getEmbeddingOutput([sampleX])[0]

            # Get Prediction for attribution
            prediction, predictionScore = getPrediction(inputStr)
            ys = np_utils.to_categorical(prediction, num_classes = nb_classes)

            #relevances = de.explain('grad*input', targetTensor * ys, inputTensor, sampleX)
            #relevances = de.explain('saliency', targetTensor * ys, inputTensor, sampleX)
            #relevances = de.explain('intgrad', targetTensor * ys, inputTensor, sampleX)
            #relevances = de.explain('deeplift', targetTensor * ys, inputTensor, sampleX)
            relevances = de.explain('elrp', preSoftmax * ys, embeddingTensor, embeddingOutput)
            #relevances = de.explain('occlusion', targetTensor * ys, inputTensor, sampleX)
    #         print ("Feature Relevances Generated...")

            wordList, wordRelevanceList, normalizedRelevanceList = getWordsAndRelevances(sampleX, relevances, prediction[0])
        #     showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList)

            relevanceScores = [-1.00 * scr for scr in wordRelevanceList]
            htmlHeatMap = html_heatmap(wordList, relevanceScores)
    #         print ("\nLRP heatmap:")    
            display(HTML(htmlHeatMap))

Prediction =  replacecard
Actual =  outofdomain


Prediction =  outofdomain
Actual =  contentonly


Prediction =  confirmation
Actual =  reportlostcard


Prediction =  outofdomain
Actual =  rejection


Prediction =  outofdomain
Actual =  openinggreeting


...

Prediction =  reportlostcard
Actual =  contentonly


Prediction =  thankyou
Actual =  closinggreeting
ok 0.5
ok -0.9747180938720703


Prediction =  contentonly
Actual =  rejection


Prediction =  outofdomain
Actual =  closinggreeting
glad 0.5
glad -0.7513898611068726


Prediction =  rejection
Actual =  thankyou
